# Web search with RAG

The objective on this notebook is to use RAG strategies to better retrieves on price web search.

The previous notebook can be found: [`experiments/finding-prices.ipynb`](finding_prices.ipynb).

---

Importing packages:

In [1]:
from dotenv import load_dotenv

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.messages import SystemMessage, HumanMessage

In [2]:
from os import getenv

In [3]:
%cd ..

/home/gabael/projetos/precificador


In [4]:
from src.rag.web import web_search

USER_AGENT environment variable not set, consider setting it to identify your requests.


---

Configurations:

In [5]:
PRODUCT = "ar condicionado"

In [6]:
load_dotenv()

True

In [7]:
EMBEDDING_MODEL = str(getenv("EMBEDDING_MODEL"))
OLLAMA_URL = str(getenv("OLLAMA_URL"))

In [8]:
LLM_MODEL = str(getenv("LLM_MODEL"))

In [9]:
PROMPT_RAG = "rlm/rag-prompt"

---

First, we gonna use the web search function to create a document!

In [10]:
product_search = web_search(PRODUCT)

print(product_search)


Ar condicionado com o menor preço | Zoom CategoriasCategoriasCelularesCelularesiPhoneiPhone 16iPhone 15iPhone 14iPhone 13iPhone 12iPhone 11Celulares SamsungGalaxy S25Galaxy S24Galaxy S23Galaxy S22Galaxy A56Galaxy A54Celulares MotorolaMoto GMoto ECelulares XiaomiXiaomi RedmiCelulares RealmeVer mais em celularesTVsTVsTV 32 polegadasTV 43 polegadasTV 50 polegadasTV 55 polegadasTV 65 polegadasTV 4KTV SamsungTV LGVer mais em TVsLavadorasLavadorasLavadora BrastempLavadora ElectroluxLavadora PanasonicLavadora ConsulLava e SecaLava e Seca LGLava e Seca SamsungNotebooksNotebooksNotebook  SamsungNotebook  DellNotebook  LenovoNotebook  AcerNotebook  AsusNotebook  GamerNotebook  i7Notebook  i5MacbookMacbook AirMacbook ProVer mais em notebooksGeladeirasGeladeirasGeladeira ElectroluxGeladeira BrastempGeladeira PanasonicGeladeira ConsulGeladeira InverseGeladeira Frost FreeGeladeira DuplexGeladeira InoxVer mais em geladeirasAr-CondicionadoAr-CondicionadoAr-condicionado 7.500 BTUsAr-condicionado 9.000

In [11]:
# Let's verify how many tokens we have

len(product_search.split())

29800

In [12]:
product_document = Document(product_search)

product_document

Document(metadata={}, page_content='\nAr condicionado com o menor preço | Zoom CategoriasCategoriasCelularesCelularesiPhoneiPhone 16iPhone 15iPhone 14iPhone 13iPhone 12iPhone 11Celulares SamsungGalaxy S25Galaxy S24Galaxy S23Galaxy S22Galaxy A56Galaxy A54Celulares MotorolaMoto GMoto ECelulares XiaomiXiaomi RedmiCelulares RealmeVer mais em celularesTVsTVsTV 32 polegadasTV 43 polegadasTV 50 polegadasTV 55 polegadasTV 65 polegadasTV 4KTV SamsungTV LGVer mais em TVsLavadorasLavadorasLavadora BrastempLavadora ElectroluxLavadora PanasonicLavadora ConsulLava e SecaLava e Seca LGLava e Seca SamsungNotebooksNotebooksNotebook  SamsungNotebook  DellNotebook  LenovoNotebook  AcerNotebook  AsusNotebook  GamerNotebook  i7Notebook  i5MacbookMacbook AirMacbook ProVer mais em notebooksGeladeirasGeladeirasGeladeira ElectroluxGeladeira BrastempGeladeira PanasonicGeladeira ConsulGeladeira InverseGeladeira Frost FreeGeladeira DuplexGeladeira InoxVer mais em geladeirasAr-CondicionadoAr-CondicionadoAr-condici

Now, let's create a vector store

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents([product_document])

In [14]:
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL, base_url=OLLAMA_URL)

In [15]:
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [16]:
vectorstore.similarity_search("preço de ar condicionado", k=10)

[Document(id='ed2f2ce6-9bc9-4891-ada3-cae998db1a75', metadata={}, page_content='elétrica.Qual é o consumo mensal de energia do ar-condicionado portátil?De acordo com a nossa calculadora, o consumo mensal do ar-condicionado portátil é de 23,2 kWh/mês, o que resulta em um gasto de R$ 74,34 por mês considerando a tarifa de R$ 0,693 por kWh. O aparelho fica ligado durante 15 horas ao longo de 22 dias.Que fabricantes de ar condicionado são os principais?LG, Gree, Philco, Springer Midea, Samsung são os principais fabricantes de ar condicionado.Qual o valor para comprar ar condicionado?O valor para comprar ar condicionado pode variar de acordo com as características de cada produto. Mas, de maneira geral, o preço de ar condicionado fica entre R$ 1.379,00 e R$ 4.169,55.PublicidadeFiltros1Categoria: Ar-CondicionadoPreços+1000 produtos nesta faixa de preçoaté R$ 3.150 (+100)R$ 3.150 a R$ 7.650 (+100)mais de R$ 7.650 (+100)atéIrSelos ZoomIndicaProdutos indicados por Especialistas Zoom. (44)Indica

For finish, let's use this vector store to retrieve the price from the product!

In [17]:
prompt_system = SystemMessage(content=f"Você é um assistente para encontrar os menores preços de {PRODUCT}. Use as seguintes ferramentas de RAG para responder a pergunta. Use três sentenças no máximo e procure manter a resposta concisa. Por fim, faça uma lista de lojas e o melhor preço de cada uma.")

In [18]:
context = "\n\n".join(document.page_content for document in vectorstore.similarity_search(f"Preço de {PRODUCT}", k = 10))

In [19]:
prompt_context = SystemMessage(content=f"Contexto: {context}")

In [20]:
prompt_correct_question = HumanMessage(f"Qual o preço de {PRODUCT}, me diga onde comprar!")

Let's use a LLM to test it!

In [21]:
chat = ChatOllama(model=LLM_MODEL, base_url=OLLAMA_URL)

In [24]:
message_correct = chat.invoke([prompt_system, prompt_context, prompt_correct_question])

In [25]:
print(message_correct.content)

O preço de um ar condicionado pode variar muito dependendo do modelo, tamanho e características, além disso, os preços podem mudar ao longo do tempo. No entanto, posso te dar algumas opções de onde comprar ar condicionado em diferentes preços:

**Preço Baixo:**

* Ar condicionado portátil: R$ 59,39 (até 1x de R$ 62,52) - Ventura
* Ar condicionado Split Hi Wall: R$ 1.758,05 (até 10x de R$ 199,78) - Magazine Luiza

**Preço Médio:**

* Ar condicionado portátil: R$ 6.999,00 - Amazon
* Ar condicionado Split Hi Wall: R$ 2.699,00 (até 10x de R$ 284,11) - Magazine Luiza
* Ar condicionado Split Cassete: R$ 1.759,00 (até 10x de R$ 195,44) - Magazine Luiza

**Preço Alto:**

* Ar condicionado portátil: R$ 7.201,00 (até 10x de R$ 720,10) - Magazine Luiza
* Ar condicionado Split Hi Wall: R$ 3.325,26 (até 10x de R$ 332,53) - Extra
* Ar condicionado Split Cassete: R$ 18.269,00 (até 10x de R$ 1.826,90) - Magazine Luiza

É importante lembrar que os preços podem variar dependendo da loja e do local onde 